In [1]:
import tarfile
import ast
import gzip
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
import math
%matplotlib inline
import matplotlib.pyplot as plt
import re
import csv
import editdistance

In [2]:
import os
import json
os.chdir("..")
print(os.getcwd())

/Users/macuser/Desktop/TVTropesScraper/TVTropesScraper


In [3]:
with open("Film/Film_tropes_dataset.json", 'r', encoding='utf-8') as json_file:  
    movie_tropes_data = json.loads(json_file.read())
with open("Literature/Literature_tropes_dataset.json", 'r', encoding='utf-8') as json_file:  
    book_tropes_data = json.loads(json_file.read())

In [4]:
n1 = len(book_tropes_data)
book_tropes = set()
for title, trope_list in book_tropes_data.items():
    for trope in trope_list:
        book_tropes.add(trope)
m1 = len(book_tropes)

In [5]:
n2 = len(movie_tropes_data)
movie_tropes = set()
for title, trope_list in movie_tropes_data.items():
    for trope in trope_list:
        movie_tropes.add(trope)
m2 = len(movie_tropes)

In [6]:
book_tropes_data2 = {"{}_{}".format(k, "BOOK") : v for k, v in book_tropes_data.items()}

In [7]:
movie_tropes_data2 = {"{}_{}".format(k, "MOVIE") : v for k, v in movie_tropes_data.items()}

In [8]:
full_tropes_data = dict()

for k, v in book_tropes_data2.items():
    full_tropes_data[k] = v
    
for k, v in movie_tropes_data2.items():
    full_tropes_data[k] = v

In [9]:
from collections import defaultdict
inverted_index = defaultdict(list)

for k, v in full_tropes_data.items():
    for trope in v:
        inverted_index[trope].append(k)

In [10]:
for k, v in full_tropes_data.items():
    full_tropes_data[k] = " ".join(v)

In [11]:
title_list = list(full_tropes_data.keys())

In [12]:
"thehungergames_MOVIE" in title_list

True

In [13]:
for i, t in enumerate(title_list):
    if "hunger" in t:
        print(t)
        print(i)

thehungergames_MOVIE
2410
thehunger_MOVIE
2976
thehungergamesmockingjaypart1_MOVIE
4463
thehungergamescatchingfire_MOVIE
4596
thehungergamesmockingjaypart2_MOVIE
4728
hunger_MOVIE
5681


In [14]:
tfidf_vec = TfidfVectorizer(min_df=0.05, max_df=0.8)
movie_and_book_by_trope = tfidf_vec.fit_transform([v for _,v in full_tropes_data.items()]).toarray()

In [15]:
ranking = []
for i in range(len(movie_and_book_by_trope)):
    ranking.append((np.dot(movie_and_book_by_trope[i], movie_and_book_by_trope[2410]), i))

In [16]:
title_list[6015]

'manofsteel_MOVIE'

In [17]:
sorted(ranking, key=lambda x : x[0], reverse=True)

[(0.9999999999999996, 2410),
 (0.4662196232211865, 6015),
 (0.45748500615612175, 217),
 (0.45748500615612175, 1175),
 (0.4439180167953399, 3423),
 (0.44215486123616277, 4893),
 (0.41668046379668267, 2937),
 (0.41230893498975707, 4066),
 (0.41011037496284275, 3355),
 (0.4088735006056448, 3026),
 (0.4055544806574356, 603),
 (0.4055544806574356, 1121),
 (0.40540901358755316, 4264),
 (0.40205119545853385, 3439),
 (0.40083856747613594, 2510),
 (0.40065571191701443, 4596),
 (0.39927784587706316, 1584),
 (0.3938961862000741, 5133),
 (0.3937126838800982, 3352),
 (0.39330230300598146, 196),
 (0.39330230300598146, 4650),
 (0.3921054490673329, 3901),
 (0.3905122069633762, 3373),
 (0.38776292560413517, 2086),
 (0.38776292560413517, 3569),
 (0.38776292560413517, 5077),
 (0.3847899861798759, 4968),
 (0.3810932320380491, 5487),
 (0.3791790668725392, 1073),
 (0.37821615220023186, 5891),
 (0.37642058460629696, 5032),
 (0.37631049049981663, 2347),
 (0.3754606255553042, 3357),
 (0.3744769604508615, 2677)

## SEPARATE BOOK AND MOVIE QUERY

In [18]:
list(movie_tropes_data.keys())[:10]

['drumline',
 'antoniasline',
 'damienomenii',
 'bandits',
 'roadhouse',
 'enemyatthegates',
 'whatseatinggilbertgrape',
 'fatherofthebride1991',
 'agnesofgod',
 'sophieschollthefinaldays']

In [19]:
list(book_tropes_data.keys())[:10]

['affinity',
 'sentencedtoprism',
 'nostromo',
 'aroomwithaview',
 'revolutionaryroad',
 'terminalworld',
 'thehellboundheart',
 'thebrokensword',
 'armor',
 'alongfortheride']

In [20]:
with open('movie_s2f.pickle', 'rb') as handle:
    movie_s2o = pickle.load(handle)

In [21]:
unused = set(movie_s2o) - set(movie_tropes_data.keys())
for u in unused:
    del movie_s2o[u]

In [22]:
with open('book_titles_s2f.pickle', 'rb') as handle:
    book_s2o = pickle.load(handle)

In [23]:
unused = set(book_s2o) - set(book_tropes_data.keys())
for u in unused:
    del book_s2o[u]

In [24]:
from typing import Dict, List

In [25]:
def invert_dict(d: Dict[str, str]):
    retval = dict()
    for k, v in d.items():
        retval[v] = k
    return retval

In [26]:
movie_s2o_inverted = invert_dict(movie_s2o)
book_s2o_inverted = invert_dict(book_s2o)

In [27]:
import editdistance

In [95]:
def get_most_similar_word(wordlist: List[str], word: str):
    min_val = 99999
    retval = None
    for w in wordlist:
        dist = editdistance.eval(w, word)
        if dist < min_val:
#             print("Swapping {} for {}".format(retval, w))
            
#             print("{} is the edit distance between {} and {}".format(dist, w, word))
            min_val = dist
            retval = w
    return retval

In [31]:
def get_similar_movies(title: str, translator: Dict[str, str]):
    if title not in translator:
        print("Could not find title: {}".format(title))
        most_similar_word = get_most_similar_word(translator.keys(), title)
        print("Querying database for: {}".format(most_similar_word))
        title = translator[most_similar_word]

In [85]:
get_similar_movies("The Avengers 2012", movie_s2o_inverted)

Could not find title: The Avengers 2012
Swapping None for drumline
Swapping drumline for stalingrad
Swapping stalingrad for the master
Swapping the master for the prince & me
Swapping the prince & me for the hunger games
Swapping the hunger games for the avengers
Querying database for: the avengers


In [33]:
all_tropes = list(book_tropes | movie_tropes)

In [34]:
datasets = {"m"}

In [35]:
books = list(book_tropes_data.keys())
movies = list(movie_tropes_data.keys())

In [36]:
from collections import defaultdict

inverted_index_books = defaultdict(list)
for book, trope_list in book_tropes_data.items():
    for trope in trope_list:
        inverted_index_books[trope].append(book)

inverted_index_movies = defaultdict(list)
for movie, trope_list in movie_tropes_data.items():
    for trope in trope_list:
        inverted_index_movies[trope].append(movie)

In [37]:
datasets = [movie_tropes_data, book_tropes_data]
inverted_indices = [inverted_index_movies, inverted_index_books]
translators = [movie_s2o_inverted, book_s2o_inverted]

In [152]:
def doc_norm(tropes_data, 
             inverted_index, 
             idf: str=None):
    if idf == "inverse":
        f = lambda trope: (1.0 / len(inverted_index[trope])) **2
    elif idf == "log":
        f = lambda trope: (1.0/(1+np.log(len(inverted_index[trope]))))**2
    elif idf is None:
        f = lambda trope: 1
    else:
        raise Exception("Invalid IDF")
                                                                                
    norms = defaultdict(int)
    for document, trope_list in tropes_data.items():
        for trope in trope_list:
            norms[document] += f(trope)
#             0.1*(norms[document]) + 2
        norms[document] = 0.5*(norms[document]) + 2
    
    return norms

In [122]:
def get_idf_func(inverted_indices: List[Dict[str, str]], idf: str):
    if idf == "inverse":
        return lambda trope: (1.0 / len(inverted_indices[0][trope])) * (1.0 / len(inverted_indices[1][trope]))
    elif idf == "log":
        return lambda trope: (1.0/(1+np.log(len(inverted_indices[0][trope])))) * (1.0/(1+np.log(len(inverted_indices[1][trope]))))
    elif idf is None:
        return lambda trope: 1
    else:
        raise Exception("Invalid IDF")

In [153]:
def find_relevant(datasets: List[Dict],
                  inverted_indices: List[Dict],
                  query: str, 
                  input_category: str,
                  result_category: str,
                  normalize: bool=True, 
                  idf:str=None,
                  min_df:int=0
                 ):
    idx = {"movie": 0, "book": 1}
    
    dataset = datasets[idx[input_category]]
    inverted_index = inverted_indices[idx[input_category]]
    
    f = get_idf_func(inverted_indices=inverted_indices, idf=idf)
    
    if query not in dataset:
        print("Could not find title: {}".format(query))
        most_similar_word = get_most_similar_word(dataset.keys(), query)
        print("Querying database for: {}".format(most_similar_word))
        query = most_similar_word

    query_vec = dataset[query]
    
    doc_scores = defaultdict(int)  
    trope_contributions = defaultdict(dict)
    
    for trope in query_vec:
        if len(inverted_indices[0][trope]) < min_df or len(inverted_indices[1][trope]) < min_df:
            continue
            
        postings = inverted_indices[idx[result_category]][trope]
        for doc in postings:
            weight_update = f(trope)
            doc_scores[doc] += weight_update
            trope_contributions[doc][trope] = weight_update

            
    if normalize:
        norms = doc_norm(datasets[idx[result_category]], 
                 inverted_indices[idx[result_category]], 
                 idf=idf)
        for d in doc_scores:
            if norms[d] != 0:
                doc_scores[d] /= norms[d]
    
    doc_idx_scores = sorted(doc_scores.items(), key=lambda x:x[1], reverse=True)
    doc_scores = [(doc, score) for doc, score in doc_idx_scores if score > 0]
    
    return doc_scores[:10], trope_contributions
    

In [156]:

titles, contributions = find_relevant(datasets=datasets,
                                      inverted_indices=inverted_indices,
                                      query="I know why the caged bird sings", 
                                      input_category="book",
                                      result_category="movie",
                                      min_df=5,
                                      normalize=True,
                                      idf="log"
                                     )

for title in titles:
    important_tropes = sorted(contributions[title[0]].items(), key= lambda x : x[1], reverse=True)
    important_tropes = [trope for trope, score in important_tropes if score > 0]
    print(title[0], important_tropes[:10])
    print("\n")


Could not find title: I know why the caged bird sings
Querying database for: iknowwhythecagedbirdsings
bastardoutofcarolina ['DeepSouth', 'ParentalSubstitute', 'ParentalAbandonment', 'WouldHurtAChild', 'DisappearedDad']


tigerland ['TheHedonist', 'DeepSouth', 'DeliberateValuesDissonance', 'BerserkButton']


themissing ['RapeAsBackstory', 'DeliberateValuesDissonance', 'DisappearedDad']


rabbitprooffence ['ButNotTooBlack', 'DeliberateValuesDissonance', 'DisappearedDad']


deucebigalowmalegigolo ['ButNotTooBlack', 'UnusualEuphemism', 'GroinAttack']


deucebigaloweuropeangigolo ['ButNotTooBlack', 'UnusualEuphemism', 'GroinAttack']


whatslovegottodowithit ['ParentalAbandonment', 'DisappearedDad', 'GroinAttack', 'BerserkButton']


wintersbone ['BigSisterInstinct', 'ParentalAbandonment', 'DisappearedDad']


fourbrothers ['InSeriesNickname', 'TheCasanova', 'ParentalAbandonment', 'EstablishingCharacterMoment', 'BerserkButton']


ruthlesspeople ['SouthernBelle', 'EstablishingCharacterMoment',

astarisborn ['Jerkass', 'MeaningfulEcho', 'FourthDateMarriage', 'PygmalionPlot', 'TheAlcoholic', 'ForgottenTrope', 'SuicideBySea', 'AwardBaitSong', 'IWantMyBelovedToBeHappy', 'SettingUpdate']

driven ['Jerkass', 'AwardBaitSong']

cominghome ['SuicideBySea', 'DrivenToSuicide']

patchadams ['Jerkass', 'AwardBaitSong', 'DrivenToSuicide']

thevow ['Jerkass', 'IWantMyBelovedToBeHappy']

broadwaydannyrose ['MeaningfulEcho', 'TheAlcoholic', 'DrivenToSuicide']

augustrush ['Jerkass', 'MeaningfulEcho', 'SettingUpdate']

crazyheart ['Jerkass', 'TheAlcoholic']

anaffairtoremember ['FourthDateMarriage', 'IWantMyBelovedToBeHappy']

poeticjustice ['Jerkass', 'TheAlcoholic']